In [ ]:
#######################################
# Auto-reload development imports
#######################################

%load_ext autoreload
%autoreload 2

In [ ]:
#######################################
#imports
#######################################

import os, sys
from mountainlab_pytools import mlproc as mlp
from jp_ephys_viz import ephys_viz_v1

In [ ]:
dsopts=[]
dsopts.append(dict(
    id='synth_tetrode_30min',
    num_channels=4,
    duration=60*30,
    average_snr=10
))
dsopts.append(dict(
    id='synth_tetrode_120min',
    num_channels=4,
    duration=60*120,
    average_snr=10
))
dsopts.append(dict(
    id='synth_16ch_30min',
    num_channels=16,
    duration=60*30,
    average_snr=10
))

In [ ]:
#######################################
# Create output directory
#######################################

datasets_dir=os.getcwd()+'/datasets'
if not os.path.exists(datasets_dir):
    print('Creating directory: {}'.format(datasets_dir))
    os.mkdir(datasets_dir)
else:
    print('Directory already exists: {}'.format(datasets_dir))

In [ ]:
import json
def synthesize_dataset(opts,output_directory):
    if not os.path.exists(output_directory):
        print('Creating directory: {}'.format(output_directory))
        os.mkdir(output_directory)
    else:
        print('Directory already exists: {}'.format(output_directory))
    mlp.addProcess(
        'ephys.synthesize_random_firings',
        dict(),
        dict(
            firings_out=output_directory+'/firings_true.mda'
        ), 
        dict(
            duration=opts['duration']
        ),
        dict()
    )
    mlp.addProcess(
        'ephys.synthesize_random_waveforms',
        dict(),
        dict(
            waveforms_out=output_directory+'/waveforms_true.mda',
            geometry_out=output_directory+'/geom.csv'
        ), 
        dict(
            M=opts['num_channels'],
            average_peak_amplitude=10*opts['average_snr']
        ),
        dict()
    )
    mlp.addProcess(
        'ephys.synthesize_timeseries',
        dict(
            firings=output_directory+'/firings_true.mda',
            waveforms=output_directory+'/waveforms_true.mda'
        ),
        dict(
            timeseries_out=output_directory+'/raw.mda'
        ), 
        dict(
            duration=opts['duration'],
            waveform_upsamplefac=13,
            noise_level=10
        ),
        dict()
    )
    params=dict(
        samplerate=30000,
        spike_sign=1
    )
    with open(output_directory+'/params.json','w') as f:
        json.dump(params,f)

In [ ]:
P=mlp.initPipeline()

In [ ]:
for dso in dsopts:
    with P:
        synthesize_dataset(dso,datasets_dir+'/'+dso['id'])